# Variational Quantum Linear Solver
*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*
## Background

System of linear equations is a basic yet extremely useful tool in mathematics. An example is that in economics, you can model the economy using linear equations. Also, it provides simple estimation for large system of non-linear systems. Hence solving system of linear equations is an important task.

Variational Quantum Linear Solver (VQLS) is a variational quantum algorithm for solving system of linear equations. It's a classical-quantum hybrid algorithm that can run on recent Noisy Intermediate-Scale Quantum (NISQ) devices. To be more specific, given a matrix $A$ and a vector $\boldsymbol{b}$, our goal is to find a vector $\boldsymbol{x}$ so that $A \boldsymbol{x} = \boldsymbol{b}$. Using VQLS, we can obtain a quantum state that is proportional to $\boldsymbol{x}$, i.e. a normalised vector $|x\rangle = \frac{\boldsymbol{x}}{\lVert \boldsymbol{x} \rVert_2}$.

## Model Principle

Solving linear equations in the quantum setting is different to the general setting due to the requirement in quantum computing that we can only apply unitary operators to a quantum state. For the input matrix $A$, we need to decompose it to a linear combination of unitary operators $A = \sum_n c_n A_n$ where each $A_n$ is a unitary operator. For the input vector $\boldsymbol{b}$, we need to assume that it's a quantum state that can be prepared by unitary operator $U$, i.e. $U|0\rangle = |b\rangle$.

![VQLS](vqls.png)

We can see that the algorithm consists of two parts. On a quantum computer, we prepare a parameterized quantum circuit (PQC) $V(\alpha)$ and compute the loss function $C(\alpha)$, then on a classical computer, we minimize parameters $\alpha$ until the loss function is below a certain threshold, denoted as $\gamma$. At the end, we output the target quantum state $|x\rangle$. The main idea behind the algorithm is that PQC $V(\alpha)$ gives us a quantum state $|\psi(\alpha)\rangle$, circuit $F(A)$ then computes how similar $A|\psi(\alpha)\rangle$ and $|b\rangle$ are, which is what the loss function $C(\alpha)$ is measuring. When the loss is small, $A|\psi(\alpha)\rangle$ and $|b\rangle$ are very close, it means $|\psi(\alpha)\rangle$ and the target $|x\rangle$ are very close, so we output $|\psi(\alpha)\rangle$ as an approximation to $|x\rangle$.

## Paddle Quantum Implementation

We use the `Circuit` class in Paddle Quantum and optimizer in Paddle Paddle to implement VQLS. For the quantum part, we use built-in `complex_entangled_layer` ansatz to build our PQC $V(\alpha)$. To compute the loss function, we use Hadamard Test and Hadamard-Overlap Test which utilizes `oracle` gate to implement the controlled-$A_n$ gates. For the classical optimization part, we used Adam optimizer to minimize the loss function.

User can use toml file to specify the input to the algorithm, matrix $A$ and vector $\boldsymbol{b}$, stored as '.npy' files. You can run the following code to randomly generate a $n\times n$ matrix $A$ and vector $\boldsymbol{b}$:

In [19]:
n = 5

import numpy as np


np.random.seed(1)
A = np.zeros([n, n], dtype="complex64")
b = np.zeros(n, dtype="complex64")
for i in range(n):
    for j in range(n):
        x = np.random.rand() * 10
        y = np.random.rand() * 10
        A[i][j] = complex(x, y)
    x = np.random.rand() * 10
    y = np.random.rand() * 10
    b[i] = complex(x, y)
np.save("./A.npy", A)
np.save("./b.npy", b)
print("Here is a randomly generated A:")
print(A)
print("Here is a randomly generated b:")
print(b)


Here is a randomly generated A:
[[4.1702199e+00+7.203245j   1.1437482e-03+3.0233257j
  1.4675589e+00+0.9233859j  1.8626021e+00+3.4556072j
  3.9676747e+00+5.3881674j ]
 [2.0445225e+00+8.781175j   2.7387592e-01+6.704675j
  4.1730480e+00+5.5868983j  1.4038694e+00+1.9810148j
  8.0074453e+00+9.682616j  ]
 [8.7638912e+00+8.946067j   8.5044211e-01+0.39054784j
  1.6983042e+00+8.781425j   9.8346835e-01+4.2110763j
  9.5788956e+00+5.3316526j ]
 [6.8650093e+00+8.346256j   1.8288277e-01+7.5014434j
  9.8886108e+00+7.4816566j  2.8044400e+00+7.892793j
  1.0322601e+00+4.4789352j ]
 [2.8777535e+00+1.3002857j  1.9366957e-01+6.7883554j
  2.1162813e+00+2.6554666j  4.9157314e+00+0.5336254j
  5.7411761e+00+1.4672858j ]]
Here is a randomly generated b:
[4.191945 +6.852195j  3.1342418+6.9232264j 6.9187713+3.1551564j
 9.085955 +2.9361415j 5.8930554+6.9975834j]


User can specify the parameters of the VQLS in the toml file. They are `depth`, `iterations`, `LR` and `gamma`, which correspond to the number of layer in the PQC $V(\alpha)$, number of iterations of the optimizer, learning rate of the optimizer and threshold of the loss function to end optimization early. By entering `python vqls.py --config config.toml` one could solve the linear system. Here we present an example of an online demo. First, define the content of the configuration file as follows, user can try out different settings by changing the parameters of `test_toml`:

In [20]:
test_toml = r"""
# The path of the input matrix A. It should be a .npy file.
A_dir = './A.npy'
# The path of the input vector b. It should be a .npy file.
b_dir = './b.npy'
# The depth of the quantum ansatz circuit.
depth = 4
# Number optimization cycles.
iterations = 200
# The learning rate of the optimizer.
LR = 0.1
# Threshold for loss value to end optimization early, default is 0.
gamma = 0
"""

Then we run the VQLS:

In [21]:
import argparse
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

import toml
import numpy as np
import paddle
from paddle_quantum.data_analysis.vqls import compute

paddle.seed(0)

if __name__ == "__main__":
    config = toml.loads(test_toml)
    A_dir = config.pop("A_dir")
    A = np.load(A_dir)
    b_dir = config.pop("b_dir")
    b = np.load(b_dir)
    result = compute(A, b, **config)

    print("Here is x that solves Ax=b:", result)
    print("This is actual b:", b)
    print("This is Ax using estimated x:", np.matmul(A, result))
    relative_error = np.linalg.norm(b - np.matmul(A, result)) / np.linalg.norm(b)
    print("Relative error: ", relative_error)
    np.save("./answer.npy", result)


 88%|████████▊ | 176/200 [02:03<00:16,  1.43it/s]

Threshold value gamma reached, ending optimization
Here is x that solves Ax=b: [ 1.3475237 -0.7860472j   0.22970617-0.88826376j -0.35111237-0.31225887j
  0.07606918+1.2138402j  -0.729564  +0.48393282j]
This is actual b: [4.191945 +6.852195j  3.1342418+6.9232264j 6.9187713+3.1551564j
 9.085955 +2.9361415j 5.8930554+6.9975834j]
This is Ax using estimated x: [4.185339 +6.8523855j 3.1297188+6.923625j  6.924285 +3.1467872j
 9.092921 +2.932943j  5.8879805+6.999589j ]
Relative error:  0.0008446976


## Citation

```
@misc{bravo-prieto2020variational,
  title = {Variational {{Quantum Linear Solver}}},
  author = {{Bravo-Prieto}, Carlos and LaRose, Ryan and Cerezo, M. and Subasi, Yigit and Cincio, Lukasz and Coles, Patrick J.},
  year = {2020},
  month = jun,
  number = {arXiv:1909.05820},
  eprint = {1909.05820},
  eprinttype = {arxiv},
  doi = {10.48550/arXiv.1909.05820}
}
```

## References

[1] “Variational Quantum Linear Solver: A Hybrid Algorithm for Linear Systems.” Carlos Bravo-Prieto, Ryan LaRose, Marco Cerezo, Yigit Subasi, Lukasz Cincio, Patrick J. Coles. arXiv:1909.05820, 2019.